# Pneumatic Decoder

Vous tombez sur un débarras recouvert d'une épaisse couche de poussière. Des caisses en bois sont appuyées contre des étagères qui occupent toute la hauteur de la pièce, et quelque part, un sifflement pneumatique s'éteint peu à peu.

## La Découverte

Sous une bâche, vous découvrez une **machine mystérieuse** dont la surface est cabossée et le laiton brille faiblement. À l'avant, il y a quelque chose qui ressemble à un affichage à sept segments, mais ceux-ci semblent être actionnés mécaniquement. Une fente étroite attend en haut, comme si elle attendait une entrée. Que pourrait-on y insérer ?

Vous avez également l'impression que la machine a besoin d'une source d'alimentation, mais il n'y a ni cordon, ni prise, seulement un **connecteur pneumatique**. L'air comprimé serait-il le moteur de la machine ? Vous cherchez votre compresseur de poche, mais vous jurez doucement : vous avez oublié l'adaptateur.

## Les Indices

Puis, à moitié enfouis sous une pile de dessins techniques jaunis, vous trouvez deux objets :

- Une **carte perforée**
- Un **schéma pneumatique**

Les lignes du schéma s'entrelacent comme des portes logiques, mais cette fois-ci, elles ne sont pas alimentées par des électrons, mais par de l'air.

## Le Défi

**Pouvez-vous reconstituer le fonctionnement de la machine ?**

---

> **Indice :** La logique booléenne peut être mise en œuvre non seulement à l'aide de fils et de transistors, mais aussi grâce à l'air et à des valves. 
> 
> La solution est écrite en **MAJUSCULES**.

On va définir les lois logiques pour chacun des types de composants

In [13]:
# Définition d´un distributeur 3/2 à commande manuelle monostable
from abc import ABC, abstractmethod
from typing import Literal, Self


class Distributeur32Monostable(ABC):

    def __init__(self, name: str) -> None:
        self._btn = False  # Position initiale
        self.port_1 = True
        self.port_3 = False
        self.name = name

    @property
    def btn(self):
        if isinstance(self._btn, bool):
            return self._btn
        return self._btn.port_2

    @property
    @abstractmethod
    def port_2(self) -> bool:
        pass

    def actionner(self):
        self._btn = True
    
    @btn.setter
    def btn(self, value: bool | Self)-> None:
        self._btn = value

    def reset(self):
        self._btn = False


class ComposantLogique(ABC):
    @property
    @abstractmethod
    def port_2(self) -> bool:...

    @property
    @abstractmethod
    def port_1_gauche(self) -> bool:...

    @property
    @abstractmethod
    def port_1_droite(self) -> bool:...

    @abstractmethod
    def reset(self) -> None:...


class Distributeur32MonostableNC(Distributeur32Monostable):

    @property
    def port_2(self) -> bool:
        # type NC
        if self.btn:
            return self.port_3
        return self.port_1

class Distributeur32MonostableNO(Distributeur32Monostable):

    @property
    def port_2(self) -> bool:
        # type NC
        if self.btn:
            return self.port_1
        return self.port_3
    



In [14]:
# verification du comportement du distributeur
distributeur = Distributeur32MonostableNO(name="Test")
print(f"--- Distributeur 3/2 Monostable NO ---")
print(f"État initial: button={distributeur.btn}, port_2={distributeur.port_2}")
distributeur.actionner()
print(f"Après action: button={distributeur.btn}, port_2={distributeur.port_2}")
distributeur.actionner()
print(f"Après deuxième action: button={distributeur.btn}, port_2={distributeur.port_2}")

print(f"\n--- Distributeur 3/2 Monostable NC ---")
distributeur = Distributeur32MonostableNC(name="Test")

print(f"État initial: button={distributeur.btn}, port_2={distributeur.port_2}")
distributeur.actionner()
print(f"Après action: button={distributeur.btn}, port_2={distributeur.port_2}")
distributeur.actionner()
print(f"Après deuxième action: button={distributeur.btn}, port_2={distributeur.port_2}")

--- Distributeur 3/2 Monostable NO ---
État initial: button=False, port_2=False
Après action: button=True, port_2=True
Après deuxième action: button=True, port_2=True

--- Distributeur 3/2 Monostable NC ---
État initial: button=False, port_2=True
Après action: button=True, port_2=False
Après deuxième action: button=True, port_2=False


On crée la liaison entre la commande et le distributeur sous-jacent

In [15]:
from typing import Literal


class BlocDistributeur:
    def __init__(self, commande: Distributeur32Monostable, distributeur: Distributeur32Monostable) -> None:
        self.commande = commande
        self.distributeur = distributeur
        self.distributeur.btn = self.commande

    @property
    def uppercase_output_upper_line(self) -> bool:
        return self.commande.port_2
    
    @property
    def uppercase_output_lower_line(self) -> bool:
        return self.distributeur.port_2
    
    @property
    def btn(self) -> bool:
        return self.commande.btn
    
    def actionner(self):
        self.commande.actionner()
        # self.distributeur._btn = any([self.commande.port_2, self.uppercase_output_upper_line])

    def deactiver(self):
        self.commande.reset()
        
    def reset(self):
        self.commande.reset()
        self.distributeur.reset()


In [16]:
distributeur = Distributeur32MonostableNC(name="Distributeur")
commande = Distributeur32MonostableNO(name="Commande")

component = BlocDistributeur(commande=commande, distributeur=distributeur)

print(f"État initial: button={component.btn}, lower_output_line={component.uppercase_output_lower_line}, upper_output_line={component.uppercase_output_upper_line}")
component.actionner()
print(f"Après action: button={component.btn}, lower_output_line={component.uppercase_output_lower_line}, upper_output_line={component.uppercase_output_upper_line}")
component.actionner()
print(f"Après deuxième action: button={component.btn}, lower_output_line={component.uppercase_output_lower_line}, upper_output_line={component.uppercase_output_upper_line}")

État initial: button=False, lower_output_line=True, upper_output_line=False
Après action: button=True, lower_output_line=False, upper_output_line=True
Après deuxième action: button=True, lower_output_line=False, upper_output_line=True


On lie les composants entre eux

In [17]:
def creer_blocs(name: str) -> BlocDistributeur:
    distributeur = Distributeur32MonostableNC(name=f"Distributeur_{name}")
    commande = Distributeur32MonostableNO(name=f"Commande_{name}")
    return BlocDistributeur(commande=commande, distributeur=distributeur)

class Line:
    def __init__(self, component: ComposantLogique | BlocDistributeur, name: str, kind: Literal['upper', 'lower', None] = None) -> None:
        self.component = component
        self.name = name
        self.kind = kind

    @property
    def port_2(self) -> bool:
        if isinstance(self.component, BlocDistributeur):
            if self.kind == 'upper':
                return self.component.uppercase_output_upper_line
            elif self.kind == 'lower':
                return self.component.uppercase_output_lower_line
        if isinstance(self.component, ComposantLogique):
            return self.component.port_2
        raise ValueError("Composant inconnu pour la ligne.")

    
    def reset(self):
        self.component.reset()
    
module_A = creer_blocs(name="A")
module_B = creer_blocs(name="B")
module_C = creer_blocs(name="C")
module_D = creer_blocs(name="D")

upper_line_A = Line(component=module_A, name="A", kind='upper')
lower_line_A = Line(component=module_A, name="A", kind='lower')
upper_line_B = Line(component=module_B, name="B", kind='upper')
lower_line_B = Line(component=module_B, name="B", kind='lower')
upper_line_C = Line(component=module_C, name="C", kind='upper')
lower_line_C = Line(component=module_C, name="C", kind='lower')
upper_line_D = Line(component=module_D, name="D", kind='upper')
lower_line_D = Line(component=module_D, name="D", kind='lower')

# Verification des lignes
print(f"\n--- Verification des lignes ---")
print(f"Ligne supérieure A: {upper_line_A.port_2}")
print(f"Ligne inférieure A: {lower_line_A.port_2}")
module_A.actionner()
print(f"Après actionnement du module A:")
print(f"Ligne supérieure A: {upper_line_A.port_2}")
print(f"Ligne inférieure A: {lower_line_A.port_2}")
module_A.actionner()
print(f"Après deuxième actionnement du module A:")
print(f"Ligne supérieure A: {upper_line_A.port_2}")
print(f"Ligne inférieure A: {lower_line_A.port_2}")


--- Verification des lignes ---
Ligne supérieure A: False
Ligne inférieure A: True
Après actionnement du module A:
Ligne supérieure A: True
Ligne inférieure A: False
Après deuxième actionnement du module A:
Ligne supérieure A: True
Ligne inférieure A: False


In [18]:
from typing import Self

ComposantPneumatique = ComposantLogique | Line

class ClapetNavetteTiroir(ComposantLogique):
    """ Distributeur 3/2 bistable se comporte comme un flip-flop. """
    def __init__(self) -> None:
        self._port_1_gauche: ComposantPneumatique | None = None
        self._port_1_droite: ComposantPneumatique | None = None

    
    @property
    def port_1_gauche(self) -> bool:
        if self._port_1_gauche is None:
            raise ValueError("Port 1 gauche non lié.")
        return self._port_1_gauche.port_2
    
    @port_1_gauche.setter
    def port_1_gauche(self, value: ComposantPneumatique) -> None:
        self._port_1_gauche = value

    @property
    def port_1_droite(self) -> bool:
        if self._port_1_droite is None:
            raise ValueError("Port 1 droite non lié.")
        return self._port_1_droite.port_2
    
    @port_1_droite.setter
    def port_1_droite(self, value: ComposantPneumatique) -> None:
        self._port_1_droite = value

    @property
    def port_2(self) -> bool:
        return self.port_1_gauche and self.port_1_droite
    
    def lier(self, *ports_1: ComposantPneumatique) -> Self:
        if len(ports_1) != 2:
            raise ValueError("Deux ports doivent être fournis pour lier le clapet navette tiroir.")
        self.port_1_gauche, self.port_1_droite = ports_1
        return self

    def reset(self):
        if self._port_1_gauche is None or self._port_1_droite is None:
            raise ValueError("Les ports 1 gauche et droite doivent être liés avant de réinitialiser.")
        self._port_1_gauche.reset()
        self._port_1_droite.reset()

In [19]:
ComposantPneumatique = ComposantLogique | Line

class ClapetShuttle(ComposantLogique):
    def __init__(self) -> None:
        self._port_1_gauche: ComposantPneumatique | None = None
        self._port_1_droite: ComposantPneumatique | None = None

    @property
    def port_1_gauche(self) -> bool:
        if self._port_1_gauche is None:
            raise ValueError("Port 1 gauche non lié.")
        return self._port_1_gauche.port_2
    
    @port_1_gauche.setter
    def port_1_gauche(self, value: ComposantPneumatique) -> None:
        self._port_1_gauche = value

    @property
    def port_1_droite(self) -> bool:
        if self._port_1_droite is None:
            raise ValueError("Port 1 droite non lié.")
        return self._port_1_droite.port_2
    
    @port_1_droite.setter
    def port_1_droite(self, value: ComposantPneumatique) -> None:
        if value is None:
            raise ValueError("Port 1 droite non lié.")
        self._port_1_droite = value

    @property
    def port_2(self) -> bool:
        return self.port_1_droite or self.port_1_gauche
    
    def lier(self, *ports_1: ComposantPneumatique) -> Self:
        assert len(ports_1) == 2, "Le clapet shuttle nécessite au moins deux entrées."
        self.port_1_gauche, self.port_1_droite = ports_1
        return self
    
    def reset(self):
        if self._port_1_droite is None or self._port_1_gauche is None:
            raise ValueError("Les ports 1 gauche et droite doivent être liés avant de réinitialiser.")
        self._port_1_droite.reset()
        self._port_1_gauche.reset()



On connecte les connectent au `uppercase lines`

In [20]:
def reset_all_modules():
    module_A.reset()
    module_B.reset()
    module_C.reset()
    module_D.reset()

In [21]:
line_a = Line(ClapetShuttle().lier(
    upper_line_B, 
    ClapetNavetteTiroir().lier(
        upper_line_A, 
        lower_line_C
        )
    ),
    name="a"
)

# Verification du comportement du stack pneumatique
# Comportement attendu:
# - Initialement, la sortie du stack est False
# - En actionnant le module B, la sortie du stack devient True
# - En actionnant le module A, la sortie du stack reste True
# - En désactivant le module B, la sortie du stack reste True
# - En actionnant le module C, la sortie du stack redevient False
reset_all_modules()
assert upper_line_A.port_2 == False
assert upper_line_B.port_2 == False
assert lower_line_C.port_2 == True
assert line_a.port_2 == False
# 
module_B.actionner()
assert upper_line_B.port_2 == True
assert line_a.port_2 == True
module_A.actionner()
assert upper_line_B.port_2 == True
assert line_a.port_2 == True
module_B.reset()
assert line_a.port_2 == True
module_C.actionner()
assert line_a.port_2 == True

reset_all_modules()

In [22]:
clapet_s2 = ClapetShuttle()
dis_bi_b_1 = ClapetNavetteTiroir()
dis_bi_b_21 = ClapetNavetteTiroir()
dis_bi_b_22 = ClapetNavetteTiroir()

dis_bi_b_21.lier(lower_line_A, upper_line_C)
dis_bi_b_22.lier(upper_line_A, lower_line_B)
dis_bi_b_1.lier(dis_bi_b_22, lower_line_C)
line_b = Line(clapet_s2.lier(dis_bi_b_21, dis_bi_b_1), name="b")

In [23]:
# Etage 2
dis_bi_c_21 = ClapetNavetteTiroir().lier(lower_line_A, upper_line_C)

dis_bi_c_22 = ClapetNavetteTiroir().lier(upper_line_B, upper_line_C)

dis_bi_c_23 = ClapetNavetteTiroir().lier(upper_line_A, lower_line_B)

# Etage 1
clapet_c_1 = ClapetShuttle().lier(dis_bi_c_21, dis_bi_c_22)

dis_bi_c_1 = ClapetNavetteTiroir().lier(dis_bi_c_23, lower_line_C)

line_c = Line(ClapetShuttle().lier(clapet_c_1, dis_bi_c_1), name="c")

In [24]:
# Definition de la stack 4
# Etage 2
dis_bi_d_21 = ClapetNavetteTiroir().lier(lower_line_A, upper_line_B)
dis_bi_d_22 = ClapetNavetteTiroir().lier(upper_line_A, upper_line_C)

# Etage 1
clapet_d_1 = ClapetShuttle().lier(upper_line_D,dis_bi_d_21)

line_d = Line(ClapetShuttle().lier(clapet_d_1, dis_bi_d_22), name="d")

In [25]:
# Definition de la stack 5
# Etage 3
dis_bi_e_31 = ClapetNavetteTiroir().lier(upper_line_A, lower_line_B)
dis_bi_e_32 = ClapetNavetteTiroir().lier(lower_line_A, upper_line_C)
dis_bi_e_33 = ClapetNavetteTiroir().lier(lower_line_A, upper_line_B)
dis_bi_e_34 = ClapetNavetteTiroir().lier(lower_line_C, upper_line_D)
dis_bi_e_35 = ClapetNavetteTiroir().lier(upper_line_A, lower_line_C)

# Etage 2
clapet_e_21 = ClapetShuttle().lier(dis_bi_e_31, dis_bi_e_32)
clapet_e_22 = ClapetShuttle().lier(dis_bi_e_33, dis_bi_e_34)
dis_bi_e_2 = ClapetNavetteTiroir().lier(dis_bi_e_35, lower_line_D)

# Etage 1
clapet_e_1 = ClapetShuttle().lier(clapet_e_21, clapet_e_22)

line_e = Line(ClapetShuttle().lier(clapet_e_1, dis_bi_e_2), name="e")

In [26]:
# Definition de la stack 6

# Etage 2
dis_bi_f_21 = ClapetNavetteTiroir().lier(upper_line_A, lower_line_B)
dis_bi_f_22 = ClapetNavetteTiroir().lier(lower_line_B, upper_line_D)
dis_bi_f_23 = ClapetNavetteTiroir().lier(lower_line_A, upper_line_C)

# Etage 1
clapet_f_1 = ClapetShuttle().lier(upper_line_B, dis_bi_f_21)
clapet_f_2 = ClapetShuttle().lier(dis_bi_f_22, dis_bi_f_23)

line_f = Line(ClapetShuttle().lier(clapet_f_1, clapet_f_2), name="f")

In [27]:
# Definition de la stack 7

# Etage 4
dis_bi_g_41 = ClapetNavetteTiroir().lier(upper_line_A, upper_line_B)
dis_bi_g_42 = ClapetNavetteTiroir().lier(upper_line_A, lower_line_C)
dis_bi_g_43 = ClapetNavetteTiroir().lier(upper_line_B, lower_line_C)
dis_bi_g_44 = ClapetNavetteTiroir().lier(lower_line_A, lower_line_B)

# Etage 3
dis_bi_g_31 = ClapetNavetteTiroir().lier(dis_bi_g_43, lower_line_D)
dis_bi_g_32 = ClapetNavetteTiroir().lier(dis_bi_g_44, upper_line_C)

# Etage 2
clapet_g_21 = ClapetShuttle().lier(upper_line_D, dis_bi_g_41)
clapet_g_22 = ClapetShuttle().lier(dis_bi_g_42, dis_bi_g_31)

# Etage 1
clapet_g_1 = ClapetShuttle().lier(clapet_g_21, clapet_g_22)

line_g = Line(ClapetShuttle().lier(clapet_g_1, dis_bi_g_32), name="g")

On lit la table des séquences des entrées à A, B, C, D

In [28]:
def reset_all_lines():
    line_a.reset()
    line_b.reset()
    line_c.reset()
    line_d.reset()
    line_e.reset()
    line_f.reset()
    line_g.reset()

In [33]:
# Mapping 7-segment → caractères
SEGMENT_TO_CHAR = {
    # a     b      c     d     e     f     g
    (True, True, True, False, True, True, True): 'A',
    (False, False, True, True, True, True, True): 'B',
    (True, False, False, True, True, True, False): 'C',
    (False, True, True, True, True, False, True): 'D',
    (True, False, False, True, True, True, True): 'E',
    (True, False, False, False, True, True, True): 'F',
    (True, False, True, True, True, True, False): 'G',
    (False, True, True, False, True, True, True): 'H',
    (False, True, True, False, False, False, False): 'I',
    (False, True, True, True, True, False, False): 'J',
    (False, False, False, False, True, True, True): 'K',
    (False, False, False, True, True, True, False): 'L',
    (True, False, False, False, True, False, True): 'M',
    (False, False, True, False, True, True, True): 'N',
    (True, True, True, True, True, True, False): 'O',
    (True, True, False, False, True, True, True): 'P',
    (True, True, True, True, False, True, True): 'Q',
    (False, False, False, False, True, True, True): 'R',
    (True, False, True, True, False, True, True): 'S',
    (False, False, False, True, True, True, True): 'T',
    (False, True, True, True, True, True, False): 'U',
    (False, False, True, True, True, False, False): 'V',
    (False, True, True, True, True, True, True): 'W',
    (False, True, True, False, False, True, True): 'X',
    (False, True, True, True, False, True, True): 'Y',
    (True, True, False, True, True, False, True): 'Z',
}

def segments_to_char(a, b, c, d, e, f, g) -> str:
    """Convertit 7 segments en caractère"""
    pattern = (a, b, c, d, e, f, g)
    return SEGMENT_TO_CHAR.get(pattern, '?')


In [34]:
from typing import Tuple

import pandas as pd


class LecteurCartePneumatique:
    def __init__(self, carte: str) -> None:
        self.name = carte
        self.carte = self.lire_carte()
        self.index = 0
    
    def lire_carte(self) -> pd.DataFrame:
        return pd.read_csv(self.name, sep='|', skipinitialspace=True).drop(columns=['Unnamed: 0', 'Unnamed: 5']).drop(index=0, axis=0)
    
    def __iter__(self):
        """Rend la classe itérable"""
        self.index = 0
        return self
    
    def __next__(self) -> Tuple[bool, bool, bool, bool]:
        if self.index >= len(self.carte):
            raise StopIteration
        
        ligne = self.carte.iloc[self.index]
        self.index += 1
        return tuple([bool(int(x.strip())) for x in ligne.values.tolist()])
    
    def activer_ou_desactiver_module(self, module: BlocDistributeur, etat: bool) -> None:
        if etat:
            module.actionner()
        else:
            module.deactiver()
            
    def executer_instruction(self) -> None:
        btn_D, btn_C, btn_B, btn_A = next(self)
        for pair in [(btn_A, module_A), (btn_B, module_B), (btn_C, module_C), (btn_D, module_D)]:
            btn, module = pair
            self.activer_ou_desactiver_module(module, btn)

    

On affiche les lettres les unes après les autres.

In [37]:
lecteur = LecteurCartePneumatique('res/card.txt')

message = []


while True:
    try:
        reset_all_modules()
        # reset_all_lines()
        lecteur.executer_instruction()
        # Utiliser après la boucle de lecture
        message.append(segments_to_char(
            line_a.port_2,
            line_b.port_2,
            line_c.port_2,
            line_d.port_2,
            line_e.port_2,
            line_f.port_2,
            line_g.port_2
        ))

    except StopIteration:
        break
print(f"\nMessage décodé : {''.join(message)}")



Message décodé : HA?????E?T?
